## Визуализация данных

На прошлом семинаре мы рассмотрели полный путь решения задачи от визуализации и предобработки до стекинга и блендинга. Пройдем его ещё раз сосредоточившись на визуализации и отборе признаков.

Понижение размерности (визуализация в 2D это частный случай понижения размерности до 2-х измерений) можно использовать и для других целей:

* Сокращение ресурсоемкости алгоритмов
* Ослабление влияния проклятия размерности и тем самым уменьшение переобучения
* Переход к более информативным признакам

## Отбор признаков

In [0]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## Метод главных компонент (Principal Component Analysis, PCA)

Выделение новых признаков путем их отбора часто дает плохие результаты, и
в некоторых ситуациях такой подход практически бесполезен. Например, если
мы работаем с изображениями, у которых признаками являются яркости пикселей,
невозможно выбрать небольшой поднабор пикселей, который дает хорошую информацию о
содержимом картинки. 

Поэтому признаки нужно как-то комбинировать. Рассмотрим метод главных компонент.

Этот метод делает два важных упрощения задачи

1. Игнорируется целевая переменная
2. Строится линейная комбинация признаков

П. 1 на первый взгляд кажется довольно странным, но на практике обычно не является
таким уж плохим. Это связано с тем, что часто данные устроены так, что имеют какую-то
внутреннюю структуру в пространстве меньшей размерности, которая никак не связана с
целевой переменной. Поэтому и оптимальные признаки можно строить не глядя на ответ.

П. 2 тоже сильно упрощает задачу, но далее мы научимся избавляться от него.

### Теория

Кратко вспомним, что делает этот метод (подробно см. в лекции).

Обозначим $X$ - матрица объекты-признаки, с нулевым средним каждого признака,
а $w$ - некоторый единичный вектор. Тогда
$Xw$ задает величину проекций всех объектов на этот вектор. Далее ищется вектор,
который дает наибольшую дисперсию полученных проекций (то есть наибольшую дисперсию
вдоль этого направления):

$$
    \max_{w: \|w\|=1} \| Xw \|^2 =  \max_{w: \|w\|=1} w^T X^T X w
$$

Подходящий вектор тогда равен собственному вектору матрицы $X^T X$ с наибольшим собственным
значением. После этого все пространство проецируется на ортогональное дополнение к вектору
$w$ и процесс повторяется.

### PCA на плоскости

Для начала посмотрим на метод PCA на плоскости для того, чтобы
лучше понять, как он устроен.

Попробуем специально сделать один из признаков более значимым и проверим, что PCA это обнаружит. Сгенерируем выборку из двухмерного гауссовского распределения. Обратите внимание, что выборка
изначально выбирается центрированной.

In [0]:
np.random.seed(314512)

data_synth_1 = np.random.multivariate_normal(
    mean=[0, 0], 
    cov=[[4, 0], 
         [0, 1]],
    size=1000)

Теперь изобразим точки выборки на плоскости и применим к ним PCA для нахождения главных компонент.
В результате работы PCA из sklearn в `dec.components_` будут лежать главные направления (нормированные), а в `dec.explained_variance_` - дисперсия, которую объясняет каждая компонента. Изобразим на нашем графике эти направления, умножив их на дисперсию для наглядного отображения их
значимости.

In [0]:
from sklearn.decomposition import PCA


def PCA_show(dataset):
    plt.scatter(*zip(*dataset), alpha=0.5)
    
    dec = PCA()
    dec.fit(dataset)
    ax = plt.gca()
    for comp_ind in range(dec.components_.shape[0]):
        component = dec.components_[comp_ind, :]
        var = dec.explained_variance_[comp_ind]
        start, end = dec.mean_, component * var
        ax.arrow(start[0], start[1], end[0], end[1],
                 head_width=0.2, head_length=0.4, fc='r', ec='r')
    
    ax.set_aspect('equal', adjustable='box')

plt.figure(figsize=(16, 8))
PCA_show(data_synth_1)

Видим, что PCA все правильно нашел. Но это, конечно, можно было сделать и просто посчитав
дисперсию каждого признака. Повернем наши данные на некоторый фиксированный угол и проверим,
что для PCA это ничего не изменит.

In [0]:
angle = np.pi / 6
rotate = np.array([
        [np.cos(angle), - np.sin(angle)],
        [np.sin(angle), np.cos(angle)],
    ])
data_synth_2 = rotate.dot(data_synth_1.T).T
plt.figure(figsize=(16, 8))
PCA_show(data_synth_2)

Ну вот, все нормально. 

Ниже пара примеров, где PCA отработал не так хорошо (в том смысле, что направления задают не очень хорошие признаки).

**Упражнение.** Объясните, почему так произошло.

### По [этой ссылке](http://setosa.io/ev/principal-component-analysis/) можно поиграться с понижением размерности трезмерных данных в интерактивном режиме - крайне рекомендую это сделать

In [0]:
from sklearn.datasets import make_circles, make_moons, make_blobs

np.random.seed(54242)
data_synth_bad = [
    make_circles(n_samples=1000, factor=0.2, noise=0.1)[0]*2,
    make_moons(n_samples=1000, noise=0.1)[0]*2,
    make_blobs(n_samples=1000, n_features=2, centers=4)[0]/5,
    np.random.multivariate_normal(
        mean=[0, 1.5], 
        cov=[[3, 1], 
             [1, 1]],
        size=1000),
]


plt.figure(figsize=(16,8))
rows, cols = 2, 2
for i, data in enumerate(data_synth_bad):
    plt.subplot(rows, cols, i + 1)
    PCA_show(data)

### Лица людей

Рассмотрим датасет с фотографиями лиц людей и применим к его признакам PCA.

Ниже изображены примеры лиц из базы, и последняя картинка - это "среднее лицо".

Подробнее о датасете можно прочитать [здесь](https://scikit-learn.org/0.22/datasets/index.html#the-olivetti-faces-dataset)

По 10 фотографий для 40 людей

In [0]:
from sklearn.datasets import fetch_olivetti_faces

faces = fetch_olivetti_faces(shuffle=True, random_state=432542)
faces_images = faces.data
faces_ids = faces.target
image_shape = (64, 64)
    
mean_face = faces_images.mean(axis=0)

plt.figure(figsize=(16, 8))
rows, cols = 2, 4
n_samples = rows * cols
for i in range(n_samples - 1):
    plt.subplot(rows, cols, i + 1)
    plt.imshow(faces_images[i, :].reshape(image_shape), interpolation='none',
               cmap='gray')
    plt.xticks(())
    plt.yticks(())
    
plt.subplot(rows, cols, n_samples)
plt.imshow(mean_face.reshape(image_shape), interpolation='none',
           cmap='gray')
plt.xticks(())
_ = plt.yticks(())

Теперь найдем главные компоненты

In [0]:
pca_reduction = PCA()
faces_images -= mean_face
pca_reduction.fit(faces_images)

plt.figure(figsize=(16, 8))
rows, cols = 2, 4
n_samples = rows * cols
for i in range(n_samples):
    plt.subplot(rows, cols, i + 1, title=f'{i}я главная компонента')
    plt.imshow(pca_reduction.components_[i, :].reshape(image_shape), interpolation='none',
               cmap='gray')
    plt.xticks(())
    plt.yticks(())

Получилось жутковато, что уже неплохо, но есть ли от этого какая-то польза?

Да, польза есть, увидим это ниже.

## Доля информации, которая содержится в первых $k$ главных компонентах, вычисляется по следующей формуле:

$$
\alpha_k = \frac{\sigma_1^2 + ... + \sigma_k^2}{\sum_{i=1}^n \sigma_i^2}, \ \ k < n
$$

## Если в первых $N$ компонентах содержится практически вся исходня информация (>90%), то это будет означать, что эффективная размерность данных равна $N$. 

## То есть можно оставить только эти первые $N$ компонент, а остальные можно отбросить

Эффективную размерность данных можно узнать из графика распределения объясненной дисперсии по главным компонентам

Это называется "Критерий крутого склона"

![alt text](https://i.ibb.co/VJDmXvh/SVslope.jpg)

### Посмотрим на значение сингулярных чисел для датасета с лицами

In [0]:
plt.figure(figsize=(10,8))
plt.plot(pca_reduction.explained_variance_, 'C1o')
plt.plot(pca_reduction.explained_variance_ratio_)
plt.title("Распределение сохрененной информации в главных компонентах")
plt.xlabel("номер главной компоненты", fontsize=15)
plt.ylabel("Процент объясненной дисперсии", fontsize=15)
plt.grid()
plt.show()

**Задание**

Посчитайте, сколько процентов информации содержится в 1, 3, 5, 10, 20 первых главных компонентах

In [0]:
# YOUR CODE HERE

Казалось бы, после отбрасывания последних компонент мы теряем информацию об исходных объектах, и вследствие этого должны получить более плохое качество работы алгоритма

Но на практике выясняется, что даже после сокращения размерности с помощью PCA **новые признаки дают более высокое качество** классификации

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

gscv_rf = GridSearchCV(RandomForestClassifier(),
                       {'n_estimators': [100, 200, 500, 800], 'max_depth': [2, 3, 4, 5]},
                       cv=5)

In [0]:
%%time
gscv_rf.fit(faces_images, faces_ids)
print(gscv_rf.best_score_)

In [0]:
%%time
gscv_rf.fit(red.transform(faces_images)[:,:100], faces_ids)
print(gscv_rf.best_score_)

Это происходит потому, что главные компоненты ортогональны друг другу, а значит новые признаки после PCA становятся ортогональны друг другу, а значит не коррелируют друг с другом. 

Именно поэтому PCA еще иногда называют **декоррелирующим преобразованием**

Напомню, что алгоритмы машинного обучения сильно страдают от наличия в данных сильно скоррелированных признаков.

Во-вторых, их можно использовать для компактного хранения данных. Для этого объекты трансформируются
в новое пространство, и из него выкидываются самые незначимые признаки.

Ниже приведены результаты сжатия более чем в 10 раз.

In [0]:
base_size = image_shape[0] * image_shape[1]

def compress_and_show(compress_ratio):
    pca_reduction = PCA(n_components=int(base_size * compress_ratio))
    pca_reduction.fit(faces_images)

    faces_compressed = pca_reduction.transform(faces_images)
    faces_restored = pca_reduction.inverse_transform(faces_compressed) + mean_face

    plt.figure(figsize=(16, 8))
    rows, cols = 2, 4
    n_samples = rows * cols
    for i in range(n_samples):
        plt.subplot(rows, cols, i + 1)
        plt.imshow(faces_restored[i, :].reshape(image_shape), interpolation='none',
                   cmap='gray')
        plt.xticks(())
        plt.yticks(())
        
compress_and_show(0.08)

И даже при сжатии в 20 раз лица остаются узнаваемыми.

In [0]:
compress_and_show(0.05)

**BONUS**

Визуализация работы **t-SNE**

![alt text](https://github.com/KellerJordan/figures/raw/master/mnist70k-tsne.gif)

## Доступное объяснение того, как работает t-SNE, на [этом сайте](https://towardsdatascience.com/an-introduction-to-t-sne-with-python-example-5a3a293108d1) и еще [на этом ресурсе](https://distill.pub/2016/misread-tsne/)

## Еще хорошие визуализации находятся в [этом репозитории](https://github.com/sophronesis/tsne_animate)

## Посмотреть, как t-SNE преобразует данные с разной геометрией можно [по этой ссылке](https://distill.pub/2016/misread-tsne/#perplexity=10&epsilon=5&demo=1&demoParams=50,2)